In [45]:
!pip install pyspark

# 0. Initialize Session

In [46]:
from pyspark.sql import SparkSession

appName = "Syntax Overview"
master = "local[*]"

spark = SparkSession.builder.master(master).appName(appName).getOrCreate()

In [47]:
spark

# 1. Reading Data
## Used Cars sales

<img src="cars.jpg" width=1256 height=353 />

In [48]:
df_cars = spark.read.csv('cars.csv', header=True)

# 2. Methods to Inspect Data
1. `.printSchema()`
2. `.explain()`
3. `.dtypes`
4. `.head(n)`
5. `.show(n)`
6. `.select()`
7. `.describe()`

### .printSchema()
Pay attention to data types!

In [49]:
df_cars.printSchema()

root
 |-- manufacturer_name: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- odometer_value: string (nullable = true)
 |-- year_produced: string (nullable = true)
 |-- engine_capacity: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- price_usd: string (nullable = true)
 |-- location_region: string (nullable = true)
 |-- number_of_photos: string (nullable = true)
 |-- up_counter: string (nullable = true)
 |-- duration_listed: string (nullable = true)



### .explain()

In [50]:
df_cars.explain()

== Physical Plan ==
FileScan csv [manufacturer_name#996,model_name#997,transmission#998,odometer_value#999,year_produced#1000,engine_capacity#1001,body_type#1002,price_usd#1003,location_region#1004,number_of_photos#1005,up_counter#1006,duration_listed#1007] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/dmitry.volkov/Desktop/HPPL/final/cars.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<manufacturer_name:string,model_name:string,transmission:string,odometer_value:string,year_...




### .dtypes

In [51]:
df_cars = spark.read.csv('cars.csv', header=True, inferSchema=True)

In [52]:
df_cars.dtypes

[('manufacturer_name', 'string'),
 ('model_name', 'string'),
 ('transmission', 'string'),
 ('odometer_value', 'int'),
 ('year_produced', 'int'),
 ('engine_capacity', 'double'),
 ('body_type', 'string'),
 ('price_usd', 'double'),
 ('location_region', 'string'),
 ('number_of_photos', 'int'),
 ('up_counter', 'int'),
 ('duration_listed', 'int')]

### .head(n)
Returns n rows as a list

In [53]:
df_cars.head(1)

[Row(manufacturer_name='Subaru', model_name='Outback', transmission='automatic', odometer_value=190000, year_produced=2010, engine_capacity=2.5, body_type='universal', price_usd=10900.0, location_region='Минская обл.', number_of_photos=9, up_counter=13, duration_listed=16)]

### .show(n)
It displays the first 20 rows by default and it also takes a number as a parameter to display the number of rows of the data.

In [54]:
df_cars.show(2)

+-----------------+----------+------------+--------------+-------------+---------------+---------+---------+---------------+----------------+----------+---------------+
|manufacturer_name|model_name|transmission|odometer_value|year_produced|engine_capacity|body_type|price_usd|location_region|number_of_photos|up_counter|duration_listed|
+-----------------+----------+------------+--------------+-------------+---------------+---------+---------+---------------+----------------+----------+---------------+
|           Subaru|   Outback|   automatic|        190000|         2010|            2.5|universal|  10900.0|   Минская обл.|               9|        13|             16|
|           Subaru|   Outback|   automatic|        290000|         2002|            3.0|universal|   5000.0|   Минская обл.|              12|        54|             83|
+-----------------+----------+------------+--------------+-------------+---------------+---------+---------+---------------+----------------+----------+---

### .select()
Selecting only a certan columns

In [55]:
df_cars = df_cars.select('manufacturer_name',
                         'model_name',
                         'odometer_value',
                         'price_usd',
                         'number_of_photos',
                         'year_produced',
                         'duration_listed')

In [56]:
df_cars.show(5)

+-----------------+----------+--------------+---------+----------------+-------------+---------------+
|manufacturer_name|model_name|odometer_value|price_usd|number_of_photos|year_produced|duration_listed|
+-----------------+----------+--------------+---------+----------------+-------------+---------------+
|           Subaru|   Outback|        190000|  10900.0|               9|         2010|             16|
|           Subaru|   Outback|        290000|   5000.0|              12|         2002|             83|
|           Subaru|  Forester|        402000|   2800.0|               4|         2001|            151|
|           Subaru|   Impreza|         10000|   9999.0|               9|         1999|             86|
|           Subaru|    Legacy|        280000|  2134.11|              14|         2001|              7|
+-----------------+----------+--------------+---------+----------------+-------------+---------------+
only showing top 5 rows



### .describe()

In [57]:
df_cars.select("manufacturer_name", "year_produced", "price_usd").describe().show()

+-------+-----------------+------------------+-----------------+
|summary|manufacturer_name|     year_produced|        price_usd|
+-------+-----------------+------------------+-----------------+
|  count|            38531|             38531|            38531|
|   mean|             null|2002.9437336170874|6639.971021255604|
| stddev|             null| 8.065730511309887|6428.152018202918|
|    min|            Acura|              1942|              1.0|
|    max|              УАЗ|              2019|          50000.0|
+-------+-----------------+------------------+-----------------+



### Checking data shape

In [58]:
rows, cols = df_cars.count(), len(df_cars.columns)
print(f'Dimension of the Dataframe is: {(rows, cols)}')

Dimension of the Dataframe is: (38531, 7)


# 2. Columns Manipulation
1. `.withColumn()`
2. `.withColumnRenamed()`
3. `.drop()`

### .withColumn()
Mutating columns, adding new ones. Method takes two parameters column name and data to add a new column to the existing data.

In [59]:
df_cars = df_cars.withColumn('price_rub', df_cars.price_usd * 75.0)

df_cars.show(3)

+-----------------+----------+--------------+---------+----------------+-------------+---------------+---------+
|manufacturer_name|model_name|odometer_value|price_usd|number_of_photos|year_produced|duration_listed|price_rub|
+-----------------+----------+--------------+---------+----------------+-------------+---------------+---------+
|           Subaru|   Outback|        190000|  10900.0|               9|         2010|             16| 817500.0|
|           Subaru|   Outback|        290000|   5000.0|              12|         2002|             83| 375000.0|
|           Subaru|  Forester|        402000|   2800.0|               4|         2001|            151| 210000.0|
+-----------------+----------+--------------+---------+----------------+-------------+---------------+---------+
only showing top 3 rows



### .withColumnRenamed()
Takes to parameters existing column name and new column name to rename the existing column.

In [60]:
df_cars = df_cars.withColumnRenamed("manufacturer_name", "manufacturer")

### .drop()

In [61]:
df_cars = df_cars.drop('price_usd')

---

# 3. Querying Data

1. `.Filter()`
2. `.Where()`
3. `Like`
4. `Distinct`
5. `Between`
6. `.GroupBy()`
7. `.OrderBy()`

### .filter()
Filter the data based on the given condition, you can also give multiple conditions using `AND(&)`, `OR(|)`, and `NOT(~)` operators.

In [62]:
df_cars.filter('manufacturer_name = "LADA" and model_name = "Priora"').show(3)

+------------+----------+--------------+----------------+-------------+---------------+---------+
|manufacturer|model_name|odometer_value|number_of_photos|year_produced|duration_listed|price_rub|
+------------+----------+--------------+----------------+-------------+---------------+---------+
|        LADA|    Priora|        115000|               3|         2008|              7| 225000.0|
|        LADA|    Priora|         73000|               8|         2011|             82| 285000.0|
|        LADA|    Priora|        106600|              14|         2015|             31| 390000.0|
+------------+----------+--------------+----------------+-------------+---------------+---------+
only showing top 3 rows



### .where()

In [63]:
df_cars.where('manufacturer_name = "LADA" and model_name like "%gus"').show(3)

+------------+----------+--------------+----------------+-------------+---------------+---------+
|manufacturer|model_name|odometer_value|number_of_photos|year_produced|duration_listed|price_rub|
+------------+----------+--------------+----------------+-------------+---------------+---------+
|        LADA|    Largus|         78932|              14|         2014|              0| 529972.5|
|        LADA|    Largus|         30500|               9|         2017|              6| 667500.0|
|        LADA|    Largus|        135350|               2|         2015|             26| 487500.0|
+------------+----------+--------------+----------------+-------------+---------------+---------+
only showing top 3 rows



**Note**

PySpark `filter()` function is used to filter the rows from RDD/DataFrame based on the given condition or SQL expression, you can also use `where()` clause instead of the `filter()` if you are coming from an SQL background, both these functions operate exactly the same.

### like

SQL `like` syntax

In [64]:
df_cars.filter('manufacturer_name like "Saab" and model_name like "9%"').show(6)

+------------+----------+--------------+----------------+-------------+---------------+---------+
|manufacturer|model_name|odometer_value|number_of_photos|year_produced|duration_listed|price_rub|
+------------+----------+--------------+----------------+-------------+---------------+---------+
|        Saab|    9 - 7X|        202208|               5|         2006|            109| 937500.0|
|        Saab|     9 - 3|        348788|              16|         2002|             21| 322500.0|
|        Saab|     9 - 3|        260000|               7|         2004|            468| 390000.0|
|        Saab|     9 - 3|        250000|               6|         2008|             98| 506250.0|
|        Saab|     9 - 3|        267000|              13|         2006|              0| 502500.0|
|        Saab|      9000|        428000|              18|         1996|             39| 126750.0|
+------------+----------+--------------+----------------+-------------+---------------+---------+
only showing top 6 r

### .between

In [65]:
df_cars.filter('year_produced between 2000 and 2007').show()

+------------+----------+--------------+----------------+-------------+---------------+---------+
|manufacturer|model_name|odometer_value|number_of_photos|year_produced|duration_listed|price_rub|
+------------+----------+--------------+----------------+-------------+---------------+---------+
|      Subaru|   Outback|        290000|              12|         2002|             83| 375000.0|
|      Subaru|  Forester|        402000|               4|         2001|            151| 210000.0|
|      Subaru|    Legacy|        280000|              14|         2001|              7|160058.25|
|      Subaru|    Legacy|        350000|               7|         2004|             73| 337500.0|
|      Subaru|  Forester|        280000|              14|         2003|             11| 517500.0|
|      Subaru|  Forester|        321000|              13|         2002|              2| 322500.0|
|      Subaru|     Justy|         49999|               8|         2001|            230| 124950.0|
|      Subaru|   Out

### .distinct()

In [66]:
df_cars.select('manufacturer').distinct().show()

+------------+
|manufacturer|
+------------+
|  Volkswagen|
|    Infiniti|
|     Peugeot|
|       Lexus|
|      Jaguar|
|       Rover|
|      Lancia|
|        Jeep|
|  Mitsubishi|
|         ГАЗ|
|         Kia|
|        Mini|
|   Chevrolet|
|       Volvo|
|       Lifan|
|     Hyundai|
|        Saab|
|        LADA|
|       Honda|
|   SsangYong|
+------------+
only showing top 20 rows



### .groupBy()

In [67]:
df_cars.groupBy('manufacturer').count().show()

+------------+-----+
|manufacturer|count|
+------------+-----+
|  Volkswagen| 4243|
|    Infiniti|  162|
|     Peugeot| 1909|
|       Lexus|  213|
|      Jaguar|   53|
|       Rover|  235|
|      Lancia|   92|
|        Jeep|  107|
|  Mitsubishi|  887|
|         ГАЗ|  200|
|         Kia|  912|
|        Mini|   68|
|   Chevrolet|  436|
|       Volvo|  721|
|       Lifan|   47|
|     Hyundai| 1116|
|        Saab|  108|
|        LADA|  146|
|       Honda|  797|
|   SsangYong|   79|
+------------+-----+
only showing top 20 rows



### `Note` lazy evaluation

In [68]:
df_cars2 = df_cars.groupBy('manufacturer').avg('price_rub')

df_cars2.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[manufacturer#1486], functions=[avg(price_rub#1437)])
   +- Exchange hashpartitioning(manufacturer#1486, 200), ENSURE_REQUIREMENTS, [id=#754]
      +- HashAggregate(keys=[manufacturer#1486], functions=[partial_avg(price_rub#1437)])
         +- Project [manufacturer_name#1036 AS manufacturer#1486, (price_usd#1043 * 75.0) AS price_rub#1437]
            +- FileScan csv [manufacturer_name#1036,price_usd#1043] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/dmitry.volkov/Desktop/HPPL/final/cars.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<manufacturer_name:string,price_usd:double>




### .orderBy()

In [69]:
df_cars.groupBy('manufacturer').avg('price_rub').orderBy('avg(price_rub)').show()

+------------+------------------+
|manufacturer|    avg(price_rub)|
+------------+------------------+
|     Москвич| 73418.41363636364|
|         ЗАЗ|102015.83928571429|
|         ВАЗ|113919.09667359668|
|      Daewoo|118245.65497737557|
|       Rover| 130068.1659574468|
|  Alfa Romeo| 201641.2572463768|
|      Lancia| 217613.1277173913|
|        Fiat|  225751.520934466|
|         УАЗ| 258234.2837837838|
|        Seat|  290101.150990099|
|         ГАЗ|       291729.8925|
|        Saab|309793.90277777775|
|     Pontiac| 327265.8035714286|
|     Peugeot|328666.32949188055|
|        Opel| 331949.4674700979|
|     Citroen|332484.46526888607|
|       Chery| 340948.3577586207|
|     Renault| 341804.9154632972|
|       Mazda| 354864.5325677711|
|        Ford| 374541.6032735776|
+------------+------------------+
only showing top 20 rows



In [71]:
df_cars.registerTempTable('df_sql')

In [72]:
query = """
SELECT *, odometer_value * 20 AS new_odometer
FROM df_sql
WHERE year_produced > 2010 AND number_of_photos >9
ORDER BY price_rub
"""

In [73]:
spark.sql(query).show()

+------------+----------+--------------+----------------+-------------+---------------+------------------+------------+
|manufacturer|model_name|odometer_value|number_of_photos|year_produced|duration_listed|         price_rub|new_odometer|
+------------+----------+--------------+----------------+-------------+---------------+------------------+------------+
|      Daewoo|     Matiz|         55000|              10|         2014|             18|          127500.0|     1100000|
|         ЗАЗ|     Lanos|        203218|              18|         2011|             19|          142500.0|     4064360|
|      Daewoo|     Nexia|        280000|              20|         2011|             92|          142500.0|     5600000|
|      Daewoo|     Nexia|         65000|              10|         2015|             18|          164925.0|     1300000|
|         ВАЗ|      2113|        160000|              19|         2011|            448|          172500.0|     3200000|
|       Geely|        CK|        150000|

In [74]:
spark.sql(query).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [price_rub#1437 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(price_rub#1437 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#841]
      +- Project [manufacturer_name#1036 AS manufacturer#1486, model_name#1037, odometer_value#1039, number_of_photos#1045, year_produced#1040, duration_listed#1047, (price_usd#1043 * 75.0) AS price_rub#1437, (odometer_value#1039 * 20) AS new_odometer#1770]
         +- Filter (((isnotnull(year_produced#1040) AND isnotnull(number_of_photos#1045)) AND (year_produced#1040 > 2010)) AND (number_of_photos#1045 > 9))
            +- FileScan csv [manufacturer_name#1036,model_name#1037,odometer_value#1039,year_produced#1040,price_usd#1043,number_of_photos#1045,duration_listed#1047] Batched: false, DataFilters: [isnotnull(year_produced#1040), isnotnull(number_of_photos#1045), (year_produced#1040 > 2010), (n..., Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/dmitry.volkov/Deskt

In [75]:
spark.stop()

---